In [1]:
import uproot

the file here is actually
```
/store/user/IDAP/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1/40000/141D3DE9-7495-7445-9344-7C65B9B80F33.root
```

In [2]:
! ls -lh 141D3DE9-7495-7445-9344-7C65B9B80F33.root

-rw-r--r--. 1 cms-jovyan users 2.2G Apr 16 11:42 141D3DE9-7495-7445-9344-7C65B9B80F33.root


In [3]:
fname = "141D3DE9-7495-7445-9344-7C65B9B80F33.root"
t = uproot.open({fname: "Events"})

size_per_key_in_MB = {}
cur_size_in_MB = t.file.source.num_requested_bytes/1000**2  # non-zero at the start (some metadata read)

for key in t.keys():
    # if not any([key.startswith(v) for v in ["Jet_", "Electron_", "Muon_", "Tau_"]]):
    #     continue
    print("reading branch", key)
    t.arrays([key])
    new_size_in_MB = t.file.source.num_requested_bytes/1000**2
    size_per_key_in_MB[key] = new_size_in_MB - cur_size_in_MB
    cur_size_in_MB = new_size_in_MB
    
size_per_key_in_MB

reading branch run
reading branch luminosityBlock
reading branch event
reading branch HTXS_Higgs_pt
reading branch HTXS_Higgs_y
reading branch HTXS_stage1_1_cat_pTjet25GeV
reading branch HTXS_stage1_1_cat_pTjet30GeV
reading branch HTXS_stage1_1_fine_cat_pTjet25GeV
reading branch HTXS_stage1_1_fine_cat_pTjet30GeV
reading branch HTXS_stage1_2_cat_pTjet25GeV
reading branch HTXS_stage1_2_cat_pTjet30GeV
reading branch HTXS_stage1_2_fine_cat_pTjet25GeV
reading branch HTXS_stage1_2_fine_cat_pTjet30GeV
reading branch HTXS_stage_0
reading branch HTXS_stage_1_pTjet25
reading branch HTXS_stage_1_pTjet30
reading branch HTXS_njets25
reading branch HTXS_njets30
reading branch nboostedTau
reading branch boostedTau_chargedIso
reading branch boostedTau_eta
reading branch boostedTau_leadTkDeltaEta
reading branch boostedTau_leadTkDeltaPhi
reading branch boostedTau_leadTkPtOverTauPt
reading branch boostedTau_mass
reading branch boostedTau_neutralIso
reading branch boostedTau_phi
reading branch boostedTau_

{'run': 0.046472999999999764,
 'luminosityBlock': 0.05075300000000027,
 'event': 0.3958949999999999,
 'HTXS_Higgs_pt': 0.04801499999999992,
 'HTXS_Higgs_y': 0.04878600000000022,
 'HTXS_stage1_1_cat_pTjet25GeV': 0.05187000000000008,
 'HTXS_stage1_1_cat_pTjet30GeV': 0.05186999999999964,
 'HTXS_stage1_1_fine_cat_pTjet25GeV': 0.053155000000000285,
 'HTXS_stage1_1_fine_cat_pTjet30GeV': 0.05315499999999984,
 'HTXS_stage1_2_cat_pTjet25GeV': 0.05187000000000008,
 'HTXS_stage1_2_cat_pTjet30GeV': 0.05187000000000008,
 'HTXS_stage1_2_fine_cat_pTjet25GeV': 0.05315499999999984,
 'HTXS_stage1_2_fine_cat_pTjet30GeV': 0.05315499999999984,
 'HTXS_stage_0': 0.04775799999999997,
 'HTXS_stage_1_pTjet25': 0.049814000000000025,
 'HTXS_stage_1_pTjet30': 0.049814000000000025,
 'HTXS_njets25': 0.04264599999999996,
 'HTXS_njets30': 0.042646000000000406,
 'nboostedTau': 0.13246899999999995,
 'boostedTau_chargedIso': 0.3901349999999999,
 'boostedTau_eta': 0.4747079999999997,
 'boostedTau_leadTkDeltaEta': 0.408439

In [4]:
print(f"all branches: {sum(size_per_key_in_MB.values())} MB")  # doesn't include initial metadata read

all branches: 2307.3440940000005 MB


In [5]:
import os
file_size = os.path.getsize(fname) / 1000**2
file_size

2311.894187

In [6]:
t.file.source.num_requested_bytes/1000**2

2309.348984

sanity check: looks like we read pretty much everything, seems good

In [7]:
import json

size_per_key_in_MB["_file_size"] = file_size

with open("branch_sizes.json", "w") as f:
    f.write(json.dumps(size_per_key_in_MB, sort_keys=True, indent=4))

## sort branches, find branches to hit 25% read

In [8]:
import pathlib

size_per_key_in_MB = json.loads(pathlib.Path("branch_sizes.json").read_text())
file_size = size_per_key_in_MB["_file_size"]

list_of_branches_to_read = []
size_of_branches = 0

# veto the metadata branch and also possibly problematic ones that could vary a lot across files
branches_to_veto = ["_file_size", "LHEPdfWeight"]

for k, v in reversed(sorted(size_per_key_in_MB.items(), key=lambda it: it[1])):
    if k in branches_to_veto:
        continue
    list_of_branches_to_read.append(k)
    size_of_branches += v
    if size_of_branches / file_size > 0.25:  # target: 25%
        break

for b in list_of_branches_to_read:
    print(f"{b}: {size_per_key_in_MB[b] / file_size:.2%}")

GenPart_pt: 1.44%
GenPart_eta: 1.42%
GenPart_phi: 1.35%
GenJet_pt: 1.26%
GenJetAK8ForJEC_pt: 1.23%
Jet_eta: 1.17%
Jet_phi: 1.16%
PSWeight: 1.13%
FatJetCHS_phi: 1.03%
FatJetCHS_eta: 1.02%
Jet_mass: 0.95%
Jet_puIdDisc: 0.94%
GenJet_eta: 0.91%
Jet_pt: 0.90%
GenJet_phi: 0.90%
GenJetAK8ForJEC_eta: 0.86%
GenJetAK8ForJEC_phi: 0.85%
FatJetCHS_mass: 0.85%
FatJetCHS_pt: 0.82%
Jet_bRegCorr: 0.76%
Jet_cRegCorr: 0.74%
JetPuppi_eta: 0.73%
Jet_muonSubtrFactor: 0.73%
GenJet_mass: 0.72%
Jet_rawFactor: 0.72%
JetPuppi_phi: 0.71%


In [9]:
list_of_branches_to_read

['GenPart_pt',
 'GenPart_eta',
 'GenPart_phi',
 'GenJet_pt',
 'GenJetAK8ForJEC_pt',
 'Jet_eta',
 'Jet_phi',
 'PSWeight',
 'FatJetCHS_phi',
 'FatJetCHS_eta',
 'Jet_mass',
 'Jet_puIdDisc',
 'GenJet_eta',
 'Jet_pt',
 'GenJet_phi',
 'GenJetAK8ForJEC_eta',
 'GenJetAK8ForJEC_phi',
 'FatJetCHS_mass',
 'FatJetCHS_pt',
 'Jet_bRegCorr',
 'Jet_cRegCorr',
 'JetPuppi_eta',
 'Jet_muonSubtrFactor',
 'GenJet_mass',
 'Jet_rawFactor',
 'JetPuppi_phi']

In [9]:
import uproot

list_of_branches_to_read = ['GenPart_pt',
 'GenPart_eta',
 'GenPart_phi',
 'GenJet_pt',
 'GenJetAK8ForJEC_pt',
 'Jet_eta',
 'Jet_phi',
 'PSWeight',
 'FatJetCHS_phi',
 'FatJetCHS_eta',
 'Jet_mass',
 'Jet_puIdDisc',
 'GenJet_eta',
 'Jet_pt',
 'GenJet_phi',
 'GenJetAK8ForJEC_eta',
 'GenJetAK8ForJEC_phi',
 'FatJetCHS_mass',
 'FatJetCHS_pt',
 'Jet_bRegCorr',
 'Jet_cRegCorr',
 'JetPuppi_eta',
 'Jet_muonSubtrFactor',
 'GenJet_mass',
 'Jet_rawFactor',
 'JetPuppi_phi']

fname = "141D3DE9-7495-7445-9344-7C65B9B80F33.root"

import os
file_size = os.path.getsize(fname) / 1000**2

In [10]:
# code for coffea schema
for branch in list_of_branches_to_read:
    print(f"_counter += ak.count_nonzero(events.{branch.replace('_', '.')})")

_counter += ak.count_nonzero(events.GenPart.pt)
_counter += ak.count_nonzero(events.GenPart.eta)
_counter += ak.count_nonzero(events.GenPart.phi)
_counter += ak.count_nonzero(events.GenJet.pt)
_counter += ak.count_nonzero(events.GenJetAK8ForJEC.pt)
_counter += ak.count_nonzero(events.Jet.eta)
_counter += ak.count_nonzero(events.Jet.phi)
_counter += ak.count_nonzero(events.PSWeight)
_counter += ak.count_nonzero(events.FatJetCHS.phi)
_counter += ak.count_nonzero(events.FatJetCHS.eta)
_counter += ak.count_nonzero(events.Jet.mass)
_counter += ak.count_nonzero(events.Jet.puIdDisc)
_counter += ak.count_nonzero(events.GenJet.eta)
_counter += ak.count_nonzero(events.Jet.pt)
_counter += ak.count_nonzero(events.GenJet.phi)
_counter += ak.count_nonzero(events.GenJetAK8ForJEC.eta)
_counter += ak.count_nonzero(events.GenJetAK8ForJEC.phi)
_counter += ak.count_nonzero(events.FatJetCHS.mass)
_counter += ak.count_nonzero(events.FatJetCHS.pt)
_counter += ak.count_nonzero(events.Jet.bRegCorr)
_counter +=

## sanity check: load file again, read these branches, ensure we hit the right fraction of the file

In [11]:
t = uproot.open({fname: "Events"})

cur_size_in_MB = t.file.source.num_requested_bytes/1000**2  # non-zero at the start (some metadata read)

t.arrays(list_of_branches_to_read)

new_size_in_MB = t.file.source.num_requested_bytes/1000**2
print(cur_size_in_MB, new_size_in_MB)

2.00489 586.540165


In [12]:
(new_size_in_MB - cur_size_in_MB) / file_size

0.2528382476528974